In [ ]:
import pandas as pd
import requests

# file = 'Внести ЛПУ в SAP.xlsx'
# path_from = 'C:/Users/osipovaa/Work documents/Работа с КА/Массовое создание ЛПУ/Исходники/'
file = 'Выгрузка КА 26.10.2020.xlsx'
path_from = 'C:/Users/osipovaa/Work documents/Работа с КА/Задачи/'
path_for_add = 'C:/Users/osipovaa/Work documents/Работа с КА/'
file_add = 'Соответствие.xlsx'

df_dop = pd.read_excel(path_for_add + file_add, header=None, names=['Регион', 'Группа сбыта'])

# df0 = pd.read_excel(path_from + file, header=None, names=['ИНН', 'GUID'], skiprows=1)
df0 = pd.read_excel(path_from + file, header=None, names=['ИНН'], sheet_name = 'Лист3')

API_KEY = '66b745ef340b66b1220623bcde71288a3b2f3d17'
url = 'https://api-fns.ru/api/multinfo'

df = pd.DataFrame()
if len(df0) > 100:
        for i in range((len(df0) // 100) + 1):
            req_INN = str(list(df0['ИНН'])[100 * i:100 * (i + 1)]).replace('[','').replace(']','')
            params = dict(key = API_KEY, req = req_INN)
            res = requests.get(url, params=params)
            df_val = pd.read_json(res.text, encoding='utf-8')
            df = pd.concat([df, df_val], axis=0).reset_index().drop(['index'], axis=1)
else:
    req_INN = str(list(df0['ИНН'])).replace('[','').replace(']','')
    params = dict(key = API_KEY, req = req_INN)
    res = requests.get(url, params=params)
    df = pd.read_json(res.text, encoding='utf-8')
del df_val

df = pd.read_csv(path_from + 'df_from_API.csv', dtype=object)

df['items'] = df['items'].apply(eval)

df_kratk = pd.DataFrame(index=range(1511), columns=['ИНН', 'Сокр.наим'])

for i in range(1511):
    df_kratk.iloc[i, 0] = df.iloc[i, 0]['ЮЛ']['ИНН']
    df_kratk.iloc[i, 1] = df.iloc[i, 0]['ЮЛ']['НаимСокрЮЛ']

musthave_column = ['ИНН', 'КПП', 'ОГРН', 'Статус', 'НаимПолнЮЛ', 'Наим1', 'Наим2', 'Наим3', 'Наим4',\
                   'КодРегион', 'Индекс', 'АдресПолн']

uniq_column = list(df.iloc[0][0]['ЮЛ'].keys())
for i in range(1, len(df)):
    new_keys = list(df.iloc[i][0]['ЮЛ'].keys())
    for j in range(len(new_keys)):
        if new_keys[j] not in uniq_column:
            uniq_column.append(new_keys[j])

index = range(len(df))
df_new = pd.DataFrame(index=index, columns=musthave_column)

for i in range(len(df)):
    for key in df.iloc[i][0]['ЮЛ'].keys():
        df_new.loc[i][key] = df.iloc[i][0]['ЮЛ'][key]

for i in range(len(df)):
    for key in df.iloc[i][0]['ЮЛ']['Адрес'].keys():
        df_new.loc[i][key] = df.iloc[i][0]['ЮЛ']['Адрес'][key]

adress = ['City', 'Street', 'House', 'Optional']
df_itog = pd.DataFrame(index=index, columns=adress)
df_itog = df_itog.fillna('')

for i in range(len(df_new)):
    sity = 0
    massiv = df_new['АдресПолн'][i].split(', ')
    for word in massiv:
        if (word.find('г.', 0, 2) >= 0 or word.find('р.п.', 0, 4) >= 0 or word.find('п.', 0, 2) >= 0 or \
               word.find('с.', 0, 2) >= 0 or word.find('пгт', 0, 3) >= 0 or word.find('ст-ца', 0, 5) >=0 \
               or word.find('дер.', 0, 4) >= 0 or word.find('сл.', 0, 3) >= 0 or word.find('аул', 0, 3) >= 0) and sity == 0:
            df_itog.loc[i,'City'] = word
            sity += 1
        elif (word.find('г.', 0, 2) >= 0 or word.find('р.п.', 0, 4) >= 0 or word.find('п.', 0, 2) >= 0 or \
                word.find('с.', 0, 2) >= 0 or word.find('пгт', 0, 3) >= 0 or word.find('ст-ца', 0, 5) >=0 \
                or word.find('дер.', 0, 4) >= 0 or word.find('сл.', 0, 3) >= 0 or word.find('аул', 0, 3) >= 0) and sity == 1:
            df_itog.loc[i,'City'] += ', ' + word
        elif word.find('ул.', 0, 3) >= 0 or word.find('пр-д', 0, 4) >= 0 or word.find('пр-кт', 0, 5) >= 0 \
            or word.find('ш.', 0, 2) >= 0 or word.find('пер.', 0, 4) >= 0 or word.find('пл.', 0, 3) >= 0 \
            or word.find('б-р', 0, 3) >= 0 or word.find('линия', 0, 5) >= 0 or word.find('спуск', 0, 5) >= 0 \
            or word.find('городок', 0, 7) >= 0 or word.find('кв-л', 0, 4) >= 0 or word.find('тер.', 0, 4) >= 0 \
            or word.find('наб.', 0, 4) >= 0 or word.find('мкр', 0, 3) >= 0 or word.find('алл.', 0, 4) >= 0 \
            or word.find('тракт', 0, 5) >= 0 or word.find(' Санаторий', 0, 10) >= 0:
            df_itog.loc[i, 'Street'] = word
        elif word.find('д.', 0, 2) >= 0 or word.find('ЗДАНИЕ', 0, 6) >= 0 or word.find('Зд.', 0, 3) >= 0:
            df_itog.loc[i, 'House'] = word
        elif word.find('лит.', 0, 4) >= 0 or word.find('кор.', 0, 4) >= 0 or word.find('стр.', 0, 4) >= 0 \
            or word.find('оф.', 0, 3) >= 0:
            df_itog.loc[i, 'Optional'] = word

df0['ИНН'] = df0['ИНН'].astype(str)

df0['ИНН'].loc[df0['ИНН'].str.len() < 10] = '0' + df0['ИНН'].loc[df0['ИНН'].str.len() < 10]

df_new['Комментарии'] = ''

df_new['КодРегион'] = df_new['КодРегион'].astype(int)

df_new = df_new.join(df_dop.set_index('Регион'), on='КодРегион')

for i in range(len(df_new)):
    if df_new['НаимПолнЮЛ'][i][160:200] != '':
        df_new['Комментарии'][i] = 'Имя > 160 знаков!'
    else:
        df_new['Наим1'][i] = str(df_new['НаимПолнЮЛ'][i][0:40])
        df_new['Наим2'][i] = str(df_new['НаимПолнЮЛ'][i][40:80])
        df_new['Наим3'][i] = str(df_new['НаимПолнЮЛ'][i][80:120])
        df_new['Наим4'][i] = str(df_new['НаимПолнЮЛ'][i][120:160])

df_new = df_new.fillna('')

df_new = df_new.join(df0.set_index('ИНН'), on='ИНН')

df_new = df_new.drop(['АдресПолн'], axis=1)

df_itog = pd.concat([df_new, df_itog], axis=1)

columns = df_itog.columns.tolist()

del columns[13]
columns.append('GUID')

df_itog = df_itog.reindex(columns=columns)

path_to = 'C:/Users/osipovaa/Work documents/Работа с КА/Массовое создание ЛПУ/'
version = 5
writer = pd.ExcelWriter(path_to + 'Egrul_parser_result_v' + str(version) + '_' + file +'.xlsx', engine='xlsxwriter')
df_itog.to_excel(writer, 'Лист1')
writer.save()

path_to = 'C:/Users/osipovaa/Work documents/Работа с КА/Массовое создание ЛПУ/'
version = 1
writer = pd.ExcelWriter(path_to + 'Сокр.наименование по задаче из репки' + str(version) + '.xlsx', engine='xlsxwriter')
df_kratk.to_excel(writer, 'Лист1')
writer.save()